In [3]:
%run "../Functions/1. Google form analysis.ipynb"
%run "../Functions/4. User comparison.ipynb"

/home/emma/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (18,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/emma/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (14,15,16,17,18,20,21,22,25,26,27,30,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/emma/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (18,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/emma/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (14,15,16,17,18,20,21,22,25,26,27,30,3

# Prepare data

## Prepare Google form data

In [4]:
setAnswerTemporalities(gform)
#gform.head()

In [5]:
# Rename columns of the Google Forms table with tags independantly of form language
columnTags = ["timestamp", "gameInterest", "gameFrequency", "age", "gender", "biologyStudy", "biologyInterest", "synthBioKnowledge", "biobrickKnowledge", "previousVersion", "previousPlay", "arcadePlay", "androidPlay", "Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8", "Q9", "Q10", "Q11", "Q12", "Q13", "Q14", "Q15", "Q16", "Q17", "Q18", "Q19", "Q20", "Q21", "Q22", "Q23", "Q24", "Q25", "Q26", "Q27", "comments", "anonymousID", "lang", "temporality"]
columnQuestions = gform.columns.values.tolist()
googleData = gform.rename(columns=dict(zip(columnQuestions, columnTags)))
#googleData.head()

In [6]:
# Replaces answers to scientific questions in the questionnaires by their values (True or False)
correctedData = googleData.copy()
for rowId in range(correctedData.shape[0]):
    # Get the correction for each subject
    playerId = correctedData.loc[rowId, "anonymousID"]
    correction = getCorrections(playerId)
    if correction.shape[1] > 0:
        # If subject has answered questionnaire
        correction = correction.rename(index=dict(zip(columnQuestions, columnTags)))
        # Replace scientific answers by their correction
        for questionId in range(27):
            questionTag = "Q" + str(questionId + 1)
            correctedData.loc[rowId, questionTag] = int(correction.loc[questionTag, "corrections" + str(rowId)])
    
#correctedData.tail(15)

In [7]:
list(zip(columnQuestions, columnTags))

[('Timestamp', 'timestamp'),
 ('Are you interested in video games?', 'gameInterest'),
 ('Do you play video games?', 'gameFrequency'),
 ('How old are you?', 'age'),
 ('What is your gender?', 'gender'),
 ('How long have you studied biology?', 'biologyStudy'),
 ('Are you interested in biology?', 'biologyInterest'),
 ('Before playing Hero.Coli, had you ever heard about synthetic biology?',
  'synthBioKnowledge'),
 ('Before playing Hero.Coli, had you ever heard about BioBricks?',
  'biobrickKnowledge'),
 ('Have you ever played an older version of Hero.Coli before?',
  'previousVersion'),
 ('Have you played the current version of Hero.Coli?', 'previousPlay'),
 ('Have you played the arcade cabinet version of Hero.Coli?', 'arcadePlay'),
 ('Have you played the Android version of Hero.Coli?', 'androidPlay'),
 ('In order to modify the abilities of the bacterium, you have to...', 'Q1'),
 ('What are BioBricks and devices?', 'Q2'),
 ('What is the name of this BioBrick?', 'Q3'),
 ('What is the name o

## For association rule mining

In [8]:
# Get only answers to scientific questions
correctedScientific = correctedData.loc[:, "Q1":"Q27"]
#correctedScientific.head()

## For clustering

### With full answers

In [9]:
# Remove timestamp and comments features
codedData = googleData.copy().drop(['timestamp', "comments"], axis=1)
#codedData.head()

In [10]:
# Code answers with integers when possible

# Define equivalences
# gameInterest
gameInterestCoding = {"A lot": 4, "Beaucoup": 4, "Enormément": 5, "Extremely": 5, "Moderately": 3, "Moyennement": 3, "Slightly": 2, "Un peu": 2, "I don't know": 3, "Je ne sais pas": 3, "Not at all": 1, "Pas du tout": 1}
# gameFrequency
gameFrequencyCoding = {"A lot": 4, "Beaucoup": 4, "Enormément": 5, "Extremely": 5, "Moderately": 3, "Moyennement": 3, "Rarely": 2, "Un peu": 2, "I don't know": 3, "Je ne sais pas": 3, "Not at all": 1, "Pas du tout": 1}
# biologyStudy
biologyStudyCoding = {"Not even in middle school": 0, "Jamais": 0, "Jamais, pas même au collège": 0, "Until the end of middle school": 1, "Jusqu'au brevet": 1, "Until the end of high school": 2, "Jusqu'au bac": 2, "Until bachelor's degree": 3, "Jusqu'à la license": 3, "At least until master's degree": 4, "Au moins jusqu'au master": 4, "I don't know": 0, "Je ne sais pas": 0}
# biologyInterest
biologyInterestCoding = {"A lot": 4, "Beaucoup": 4, "Enormément": 5, "Extremely": 5, "Moderately": 3, "Moyennement": 3, "Slightly": 2, "Un peu": 2, "I don't know": 3, "Je ne sais pas": 3, "Not at all": 1, "Pas du tout": 1}
# synthBioKnowledge
# biobrickKnowledge
previousKnowledgeCoding = {"Yes": 1, "No": 0, "I don't know": 0, "Oui": 1, 'Non': 0, "Je ne sais pas": 0}
# previousVersion
# previousPlay
# arcadePlay
# androidPlay
previousPlayCoding = {"Multiple times": 3, "A few times": 2, "Once": 1, "Yes": 1, "No": 0, "I don't know": 0, "De nombreuses fois": 3, "Quelques fois": 2, "Une fois": 1, "Oui": 1, "Non": 0, "Je ne sais pas": 0}
# lang
languageCoding = {"en": 0, "fr": 1}
# temporality
temporalityCoding = {"before": 0, "after": 1, "undefined": -5}

# Fill NaN cells
codedData["biobrickKnowledge"].fillna("I don't know", inplace = True)
codedData["arcadePlay"].fillna("I don't know", inplace = True)
codedData["androidPlay"].fillna("I don't know", inplace = True)
codedData["previousPlay"].fillna("I don't know", inplace = True)

# Replace by code
for rowId in range(codedData.shape[0]):
    codedData.loc[rowId, "gameInterest"] = gameInterestCoding[codedData.loc[rowId, "gameInterest"]]
    codedData.loc[rowId, "gameFrequency"] = gameFrequencyCoding[codedData.loc[rowId, "gameFrequency"]]
    codedData.loc[rowId, "biologyStudy"] = biologyStudyCoding[codedData.loc[rowId, "biologyStudy"]]
    codedData.loc[rowId, "biologyInterest"] = biologyInterestCoding[codedData.loc[rowId, "biologyInterest"]]
    codedData.loc[rowId, "synthBioKnowledge"] = previousKnowledgeCoding[codedData.loc[rowId, "synthBioKnowledge"]]
    codedData.loc[rowId, "biobrickKnowledge"] = previousKnowledgeCoding[codedData.loc[rowId, "biobrickKnowledge"]]
    codedData.loc[rowId, "previousVersion"] = previousPlayCoding[codedData.loc[rowId, "previousVersion"]]
    codedData.loc[rowId, "previousPlay"] = previousPlayCoding[codedData.loc[rowId, "previousPlay"]]
    codedData.loc[rowId, "arcadePlay"] = previousPlayCoding[codedData.loc[rowId, "arcadePlay"]]
    codedData.loc[rowId, "androidPlay"] = previousPlayCoding[codedData.loc[rowId, "androidPlay"]]
    codedData.loc[rowId, "lang"] = languageCoding[codedData.loc[rowId, "lang"]]
    codedData.loc[rowId, "temporality"] = temporalityCoding[codedData.loc[rowId, "temporality"]]
    

In [11]:
# One-Hot version
codedData = pd.get_dummies(codedData, prefix = ["gender", "Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8", "Q9", "Q10", "Q11", "Q12", "Q13", "Q14", "Q15", "Q16", "Q17", "Q18", "Q19", "Q20", "Q21", "Q22", "Q23", "Q24", "Q25", "Q26", "Q27"], columns = ["gender", "Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8", "Q9", "Q10", "Q11", "Q12", "Q13", "Q14", "Q15", "Q16", "Q17", "Q18", "Q19", "Q20", "Q21", "Q22", "Q23", "Q24", "Q25", "Q26", "Q27"])
#codedData.head()

In [12]:
# Split the forms according to temporality
beforeForms = codedData.copy().loc[codedData["temporality"] == 0,:]
afterForms = codedData.copy().loc[codedData["temporality"] == 1,:]
undefForms = codedData.copy().loc[codedData["temporality"] == -5,:]
defForms = codedData.copy().loc[codedData["temporality"] >= 0,:] # Either before or after
# For subjects with both before and after forms, join the two
beforeAndAfterForms = pd.merge(beforeForms, afterForms, on="anonymousID", suffixes=('_before', '_after'))
#beforeForms.head()
#afterForms.head()
#undefForms.head()
#defForms.head()
#beforeAndAfterForms.head()

In [13]:
# Remove ID feature
allForms = codedData.copy().drop("anonymousID", axis=1)
beforeForms.drop("anonymousID", axis=1, inplace = True)
afterForms.drop("anonymousID", axis=1, inplace = True)
undefForms.drop("anonymousID", axis=1, inplace = True)
defForms.drop("anonymousID", axis=1, inplace = True)
beforeAndAfterForms.drop("anonymousID", axis=1, inplace = True)

### With corrected scientific answers

In [14]:
# Remove timestamp and comments features
codedCorrectedData = correctedData.copy().drop(['timestamp', "comments"], axis=1)

In [15]:
# Fill NaN cells
codedCorrectedData["biobrickKnowledge"].fillna("I don't know", inplace = True)
codedCorrectedData["arcadePlay"].fillna("I don't know", inplace = True)
codedCorrectedData["androidPlay"].fillna("I don't know", inplace = True)
codedCorrectedData["previousPlay"].fillna("I don't know", inplace = True)

# Replace by code
for rowId in range(codedData.shape[0]):
    codedCorrectedData.loc[rowId, "gameInterest"] = gameInterestCoding[codedCorrectedData.loc[rowId, "gameInterest"]]
    codedCorrectedData.loc[rowId, "gameFrequency"] = gameFrequencyCoding[codedCorrectedData.loc[rowId, "gameFrequency"]]
    codedCorrectedData.loc[rowId, "biologyStudy"] = biologyStudyCoding[codedCorrectedData.loc[rowId, "biologyStudy"]]
    codedCorrectedData.loc[rowId, "biologyInterest"] = biologyInterestCoding[codedCorrectedData.loc[rowId, "biologyInterest"]]
    codedCorrectedData.loc[rowId, "synthBioKnowledge"] = previousKnowledgeCoding[codedCorrectedData.loc[rowId, "synthBioKnowledge"]]
    codedCorrectedData.loc[rowId, "biobrickKnowledge"] = previousKnowledgeCoding[codedCorrectedData.loc[rowId, "biobrickKnowledge"]]
    codedCorrectedData.loc[rowId, "previousVersion"] = previousPlayCoding[codedCorrectedData.loc[rowId, "previousVersion"]]
    codedCorrectedData.loc[rowId, "previousPlay"] = previousPlayCoding[codedCorrectedData.loc[rowId, "previousPlay"]]
    codedCorrectedData.loc[rowId, "arcadePlay"] = previousPlayCoding[codedCorrectedData.loc[rowId, "arcadePlay"]]
    codedCorrectedData.loc[rowId, "androidPlay"] = previousPlayCoding[codedCorrectedData.loc[rowId, "androidPlay"]]
    codedCorrectedData.loc[rowId, "lang"] = languageCoding[codedCorrectedData.loc[rowId, "lang"]]
    codedCorrectedData.loc[rowId, "temporality"] = temporalityCoding[codedCorrectedData.loc[rowId, "temporality"]]

In [16]:
# One-Hot versions of the above dataframes
codedCorrectedData = pd.get_dummies(codedCorrectedData, prefix = ["gender"], columns = ["gender"])
codedCorrectedData.head()

,gameInterest,gameFrequency,age,biologyStudy,biologyInterest,synthBioKnowledge,biobrickKnowledge,previousVersion,previousPlay,arcadePlay,...,Q25,Q26,Q27,anonymousID,lang,temporality,gender_Female,gender_Male,gender_Other,gender_Prefer not to say
0,5,5,23,3,5,1,0,0,1,0,...,1,1,1,8d352896-a3f1-471c-8439-0f426df901c1,0,1,1,0,0,0
1,3,3,28,2,3,1,0,0,1,0,...,1,1,1,7037c5b2-c286-498e-9784-9a061c778609,0,1,0,0,1,0
2,4,3,20,3,3,0,0,0,1,0,...,1,1,1,5c4939b5-425b-4d19-b5d2-0384a515539e,0,1,1,0,0,0
3,3,3,0,0,3,0,0,0,1,0,...,1,1,1,7825d421-d668-4481-898a-46b51efe40f0,0,-5,0,0,1,0
4,3,3,21,3,4,1,0,0,1,0,...,1,1,1,acb9c989-b4a6-4c4d-81cc-6b5783ec71d8,0,1,0,1,0,0


In [17]:
# Split the forms according to temporality
beforeCorrectedForms = codedCorrectedData.copy().loc[codedCorrectedData["temporality"] == 0,:]
afterCorrectedForms = codedCorrectedData.copy().loc[codedCorrectedData["temporality"] == 1,:]
undefCorrectedForms = codedCorrectedData.copy().loc[codedCorrectedData["temporality"] == -5,:]
defCorrectedForms = codedCorrectedData.copy().loc[codedCorrectedData["temporality"] >= 0,:] # Either before or after
# For subjects with both before and after forms, join the two
beforeAndAfterCorrectedForms = pd.merge(beforeCorrectedForms, afterCorrectedForms, on="anonymousID", suffixes=('_before', '_after'))
#beforeCorrectedForms.head()
#afterCorrectedForms.head()
#undefCorrectedForms.head()
#defCorrectedForms.head()
#beforeAndAfterCorrectedForms.head()

In [18]:
# Remove ID feature
allCorrectedForms = codedCorrectedData.copy().drop("anonymousID", axis=1)
beforeCorrectedForms.drop("anonymousID", axis=1, inplace = True)
afterCorrectedForms.drop("anonymousID", axis=1, inplace = True)
undefCorrectedForms.drop("anonymousID", axis=1, inplace = True)
defCorrectedForms.drop("anonymousID", axis=1, inplace = True)
beforeAndAfterCorrectedForms.drop("anonymousID", axis=1, inplace = True)

## RedMetrics

## For clustering

In [29]:
# Fetch RedMetrics data for subjects which answered the gform
allData = getAllUserVectorData( getAllResponders() )

A Jupyter Widget

"8d352896-a3f1-471c-8439-0f426df901c1"
"7037c5b2-c286-498e-9784-9a061c778609"
"5c4939b5-425b-4d19-b5d2-0384a515539e"
"7825d421-d668-4481-898a-46b51efe40f0"
"acb9c989-b4a6-4c4d-81cc-6b5783ec71d8"
"1a03bc9e-bed4-4ddd-be7f-af23b1d5eb65"
"3685a015-fa97-4457-ad73-da1c50210fe1"
"9a2e8271-a84d-4b3c-a670-2d65326ed824"
"bfdfd356-5d6f-4696-a2f1-c1dc338aa64b"
"a7936587-8b71-43b6-9c61-17b2c2b55de3"
"7ea5c999-8001-4eee-9cdf-580d0783ce5c"
"5977184a-1be2-4725-9b48-f2782dc03efb"
"e63a2fc6-50bd-4417-a712-fe688bf8e242"
"2b91e6b2-d5ed-42e1-8bf5-dc23aeaf7e6a"
"bffd0877-e77e-470a-b0f9-a9cdc6ff0a09"
"97f6ae4f-e2a6-4721-8936-f759c8867f6a"
"3ef14300-4987-4b54-a56c-5b6d1f8a24a1"
"6b5d392d-b737-49ef-99af-e8c445ff6379"
"5ecf601d-4eac-433e-8056-3a5b9eda0555"
"00dbbdca-d86c-4bc9-803c-0602e0153f68"
"3685a015-fa97-4457-ad73-da1c50210fe1"
"5977184a-1be2-4725-9b48-f2782dc03efb"
"3ef14300-4987-4b54-a56c-5b6d1f8a24a1"
"5ecf601d-4eac-433e-8056-3a5b9eda0555"
"bffd0877-e77e-470a-b0f9-a9cdc6ff0a09"
"1f27519a-971f-4e39-bac7-

In [30]:
# Put subjects as rows and features as columns
allData = allData.transpose()
#allData.head()

In [31]:
# Drop useless columns in which all rows have the same value
allData.drop(["switch", "gotomooc"], axis=1, inplace = True)
# Fill NaN with negatives
allData.fillna(-1, inplace = True)

In [36]:
# Get the subset of subjects who have answered before and after
fullProcessData = allData[allData["scorebefore"] >= 0]
fullProcessData = fullProcessData[fullProcessData["scoreafter"] >= 0]
#fullProcessData

,sessionsCount,scorebefore,scoreafter,scoreundefined,complete,configure,craft,death,equip,unequip,add,remove,gotourl,pickup,reach,restart,selectmenu,start
"""3685a015-fa97-4457-ad73-da1c50210fe1""",1.0,2.0,20.0,-1.0,0.0,0.0,16.0,54.0,78.0,10.0,76.0,6.0,3.0,15.0,10.0,0.0,0.0,1.0
"""bfdfd356-5d6f-4696-a2f1-c1dc338aa64b""",4.0,5.0,18.0,-1.0,0.0,1.0,10.0,67.0,67.0,15.0,52.0,11.0,2.0,19.0,18.0,0.0,0.0,4.0
"""a7936587-8b71-43b6-9c61-17b2c2b55de3""",2.0,4.0,13.0,-1.0,0.0,1.0,3.0,19.0,15.0,0.0,12.0,0.0,8.0,3.0,2.0,0.0,6.0,2.0
"""5977184a-1be2-4725-9b48-f2782dc03efb""",1.0,4.0,24.0,-1.0,1.0,0.0,20.0,61.0,34.0,3.0,178.0,122.0,10.0,18.0,23.0,0.0,7.0,1.0
"""e63a2fc6-50bd-4417-a712-fe688bf8e242""",1.0,10.0,23.0,-1.0,1.0,1.0,12.0,89.0,48.0,4.0,39.0,8.0,5.0,19.0,21.0,0.0,4.0,1.0
"""2b91e6b2-d5ed-42e1-8bf5-dc23aeaf7e6a""",1.0,7.0,22.0,-1.0,0.0,1.0,13.0,86.0,3.0,6.0,146.0,23.0,12.0,17.0,17.0,0.0,10.0,1.0
"""bffd0877-e77e-470a-b0f9-a9cdc6ff0a09""",2.0,8.0,22.0,-1.0,0.0,0.0,14.0,68.0,38.0,5.0,106.0,69.0,2.0,17.0,19.0,0.0,0.0,2.0
"""97f6ae4f-e2a6-4721-8936-f759c8867f6a""",3.0,3.0,25.0,-1.0,1.0,1.0,21.0,24.0,1.0,0.0,99.0,30.0,2.0,19.0,20.0,1.0,7.0,3.0
"""3ef14300-4987-4b54-a56c-5b6d1f8a24a1""",1.0,0.0,13.0,-1.0,0.0,0.0,13.0,47.0,73.0,3.0,113.0,78.0,4.0,13.0,15.0,0.0,2.0,1.0
"""6b5d392d-b737-49ef-99af-e8c445ff6379""",3.0,2.0,6.0,-1.0,0.0,0.0,26.0,53.0,108.0,1.0,102.0,73.0,3.0,20.0,23.0,0.0,0.0,3.0
